# **1. Kết nối với google drive**

In [ ]:
# kết nối với gg drive
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NLP 4+1/BTL3- BERT and GPT/pre-trained/')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/.shortcut-targets-by-id/1bs2wermM-w1hKEOcHyv-UmLHY5iag5cm/NLP 4+1/BTL3- BERT and GPT/pre-trained'

# **2. Install và import các thư viện cần thiết**

In [ ]:
!pip install transformers --quiet
!pip install keras_preprocessing --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 3.7 MB/s eta 0:00:00


In [ ]:
import os
import re
import time
import torch
import pickle
import numpy as np
import pandas as pd

from keras.models import load_model
from keras_preprocessing.sequence import pad_sequences
from transformers import GPT2LMHeadModel, BertLMHeadModel

import nltk
nltk.download('punkt')

from nltk import word_tokenize

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **3. Set các biến cục bộ và load dữ liệu**

In [ ]:
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

def preprocess(text):
    text = text.lower()
    text = decontractions(text)
    text = re.sub(r'[!“”"’#$%&\()*+,./:;<=>?@[\]^_`{|}~]', "", text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# loading
def load_file(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# **4. Load pre-trained cho các model**

## ***4.1 Model GPT***

In [ ]:
tokenizer_GPT = load_file('./GPT/tokenizer.pickle')
model_GPT = load_file('./GPT/GPT2LMHeadModel.pickle')

In [ ]:
model_GPT.resize_token_embeddings(len(tokenizer_GPT))
model_GPT = model_GPT.to(device)
if device == 'cpu':
    model_GPT.load_state_dict(torch.load('./GPT/model_GPT.pt', map_location ='cpu'))
else:
    model_GPT.load_state_dict(torch.load('./GPT/model_GPT.pt'))

In [ ]:
def GPT_answer(inp, tokenizer=tokenizer_GPT, model=model_GPT):
    inp = '<s> '+ preprocess(inp) +" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    mask = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=mask, max_length=96, pad_token_id=0, eos_token_id=50256)
    output = tokenizer.decode(output[0])
    try:
        result = re.search(r'<bot>:(.*?)</s>', output).group(1).strip()
    except:
        result = re.search(r'<bot>:\s*(.*)', output).group(1).strip()
    return result

In [ ]:
GPT_answer("do i need to exercise to benefit from vitamin c")

'no vitamin c is absorbed without exercise but do not get me wrong exercise is very desirable on its own certainly not exercise is not required to increase absorption of nutrients'

## ***4.2 Model BERT***

In [ ]:
tokenizer_BERT = load_file('./BERT/tokenizer.pickle')
model_BERT = load_file('./BERT/BertLMHeadModel.pickle')

In [ ]:
model_BERT = model_BERT.to(device)
if device == 'cpu':
    model_BERT.load_state_dict(torch.load('./BERT/model_BERT.pt', map_location ='cpu'))
else:
    model_BERT.load_state_dict(torch.load('./BERT/model_BERT.pt'))

In [ ]:
def BERT_answer(inp, tokenizer=tokenizer_BERT, model=model_BERT):
    inputs = tokenizer(preprocess(inp), max_length=120, padding='max_length', return_tensors="pt").to(device)
    outputs = model(**inputs, labels=inputs["input_ids"])

    result = []
    for word_embeds in outputs.logits[0]:
        word_embed = word_embeds
        max_idx_t = word_embed.argmax()
        max_idx = max_idx_t.item()
        word = tokenizer.convert_ids_to_tokens(max_idx)
        if word == '[PAD]' or word == '[SEP]': 
            break
        result.append(word)
    return ' '.join(result)

In [ ]:
BERT_answer('do i need to exercise to benefit from vitamin c')

'no vitamin c is absorbed without exercise but do not get me wrong wrong'

## ***4.3 Model LSTM***

In [ ]:
# LSTM (100 epoch)
tokenizer_LSTM_100 = load_file( './LSTM (100 epoch)/tokenizer.pickle')
enc_model_LSTM_100 = load_model('./LSTM (100 epoch)/enc_model.h5')
dec_model_LSTM_100 = load_model('./LSTM (100 epoch)/dec_model.h5')

# LSTM (200 epoch)
tokenizer_LSTM_200 = load_file( './LSTM (200 epoch)/tokenizer.pickle')
enc_model_LSTM_200 = load_model('./LSTM (200 epoch)/enc_model.h5')
dec_model_LSTM_200 = load_model('./LSTM (200 epoch)/dec_model.h5')

# LSTM + Attention (100 epoch)
tokenizer_LSTM_Attention_100 = load_file( './LSTM + Attention (100 epoch)/tokenizer.pickle')
enc_model_LSTM_Attention_100 = load_model('./LSTM + Attention (100 epoch)/enc_model.h5')
dec_model_LSTM_Attention_100 = load_model('./LSTM + Attention (100 epoch)/dec_model.h5')

# LSTM + Attention (200 epoch)
tokenizer_LSTM_Attention_200 = load_file( './LSTM + Attention (200 epoch)/tokenizer.pickle')
enc_model_LSTM_Attention_200 = load_model('./LSTM + Attention (200 epoch)/enc_model.h5')
dec_model_LSTM_Attention_200 = load_model('./LSTM + Attention (200 epoch)/dec_model.h5')

In [ ]:
MAXLEN_QUESTION = 32
MAXLEN_ANSWER = 64
def str_to_tokens(sentence, tokenizer):
    words = preprocess(sentence)
    tokens_list = tokenizer.texts_to_sequences([words])
    return pad_sequences(tokens_list, maxlen=MAXLEN_QUESTION,padding='post')
    
def LSTM_answer(question, with_epochs):
    if with_epochs == 100:
        tokenizer, enc_model, dec_model = tokenizer_LSTM_100, enc_model_LSTM_100, dec_model_LSTM_100
    elif with_epochs == 200:
        tokenizer, enc_model, dec_model = tokenizer_LSTM_200, enc_model_LSTM_200, dec_model_LSTM_200
    else:
        return 'Sai with_epoch rồi'

    states_values = enc_model.predict(str_to_tokens(question, tokenizer), verbose=0)
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0,0] = tokenizer.word_index['<s>']
    stop_condition = False
    generated_answer = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq]+states_values, verbose=0)
        sampled_word_index = np.argmax(dec_outputs[0,-1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                if word != '</s>':
                    generated_answer += f'{word} '
                sampled_word = word

        if sampled_word == '</s>' or len(generated_answer.split()) > MAXLEN_ANSWER:
            stop_condition = True
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = sampled_word_index
        states_values = [h,c]

    return generated_answer.strip()

def LSTM_Attention_answer(question, with_epochs):
    if with_epochs == 100:
        tokenizer, enc_model, dec_model = tokenizer_LSTM_Attention_100, enc_model_LSTM_Attention_100, dec_model_LSTM_Attention_100
    elif with_epochs == 200:
        tokenizer, enc_model, dec_model = tokenizer_LSTM_Attention_200, enc_model_LSTM_Attention_200, dec_model_LSTM_Attention_200
    else:
        return 'Sai with_epoch rồi'

    values,h,c = enc_model.predict(str_to_tokens(question, tokenizer), verbose=0)
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0,0] = tokenizer.word_index['<s>']
    stop_condition = False
    generated_answer = ''

    while not stop_condition:
        dec_outputs, h_state,c_state = dec_model.predict([empty_target_seq]+[values,h,c], verbose=0)
        sampled_word_index = np.argmax(dec_outputs[0,-1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                if word != '</s>':
                    generated_answer += f'{word} '
                sampled_word = word

        if sampled_word == '</s>' or len(generated_answer.split()) > MAXLEN_ANSWER:
            stop_condition = True
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = sampled_word_index
        h,c = h_state,c_state

    return generated_answer.strip()

print('LSTM (100 epochs):', LSTM_answer('do i need to exercise to benefit from vitamin c', with_epochs=100))
print('LSTM (200 epochs):', LSTM_answer('do i need to exercise to benefit from vitamin c', with_epochs=200))
print('LSTM + Attention (100 epochs):', LSTM_Attention_answer('do i need to exercise to benefit from vitamin c', with_epochs=100))
print('LSTM + Attention (200 epochs):', LSTM_Attention_answer('do i need to exercise to benefit from vitamin c', with_epochs=200))

LSTM (100 epochs): no
LSTM (200 epochs): no vitamin c is absorbed without exercise but do not take any of the year you should be fine
LSTM + Attention (100 epochs): certainly not matter of this also used certainly make an increase in your scar vitamins make sure to help reduce the amount of healthy 30 until it is more likely to increase the amount of a further covered in your skin with vitamin is not required to break vitamin e vitamin c is not fix
LSTM + Attention (200 epochs): no vitamin is not a absorbed or when it works but sweet but not high vitamin is not absorbed but not a wise required you may want to get me vitamin it is important that you get if it fails is not absorbed


# **5. Chat với các model**

In [ ]:
def chatbot_all(inp):
    print('USER                    :', inp)
    print('(LSTM 100)              :', LSTM_answer(inp, with_epochs=100)) 
    print('(LSTM 200)              :', LSTM_answer(inp, with_epochs=200))
    print('(LSTM + Attention 100)  :', LSTM_Attention_answer(inp, with_epochs=100)) 
    print('(LSTM + Attention 200)  :', LSTM_Attention_answer(inp, with_epochs=200)) 
    print('(BERT)                  :', BERT_answer(inp)) 
    print('(GPT)                   :', GPT_answer(inp)) 
      
chatbot_all('do i need to exercise to benefit from vitamin c')

USER                    : do i need to exercise to benefit from vitamin c
(LSTM 100)              : no
(LSTM 200)              : no vitamin c is absorbed without exercise but do not take any of the year you should be fine
(LSTM + Attention 100)  : certainly not matter of this also used certainly make an increase in your scar vitamins make sure to help reduce the amount of healthy 30 until it is more likely to increase the amount of a further covered in your skin with vitamin is not required to break vitamin e vitamin c is not fix
(LSTM + Attention 200)  : no vitamin is not a absorbed or when it works but sweet but not high vitamin is not absorbed but not a wise required you may want to get me vitamin it is important that you get if it fails is not absorbed
(BERT)                  : no vitamin c is absorbed without exercise but do not get me wrong wrong
(GPT)                   : no vitamin c is absorbed without exercise but do not get me wrong exercise is very desirable on its own certa

In [ ]:
def chatbot_with_model(name_model):
    print('bot  : May I help you')
    while True:
        inp = input('user : ')
        if inp == 'bye':
            print('bot  : bye, see you next time.')
            break
            
        if name_model == 'LSTM_100':
            print('bot  :', LSTM_answer(inp, with_epochs=100))
        elif name_model == 'LSTM_200':
            print('bot  :', LSTM_answer(inp, with_epochs=200))
        elif name_model == 'LSTM_Attention_100':
            print('bot  :', LSTM_Attention_answer(inp, with_epochs=100))
        elif name_model == 'LSTM_Attention_200':
            print('bot  :', LSTM_Attention_answer(inp, with_epochs=200))
        elif name_model == 'BERT':
            print('bot  :', BERT_answer(inp))
        elif name_model == 'GPT':
            print('bot  :', GPT_answer(inp))

chatbot_with_model(name_model='GPT')

bot  : May I help you
user : do i need to exercise to benefit from vitamin c
bot  : no vitamin c is absorbed without exercise but do not get me wrong exercise is very desirable on its own certainly not exercise is not required to increase absorption of nutrients
user : bye
bot  : bye, see you next time.
